In [37]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

In [60]:
connection_url = URL.create(
    drivername="postgresql+psycopg2",
    username="postgres",
    password="123",  
    host="localhost",
    port=5432,
)

engine = create_engine(connection_url)
conn = engine.connect()

print("Успешное подключение к PostgreSQL!")


Успешное подключение к PostgreSQL!


**Задача №1**

In [61]:
query = """
with country_sales as (
    select 
        i.billing_country, 
        sum(il.unit_price * il.quantity) as total_sales, 
        count(distinct il.track_id) as track_count
    from invoice i
    join invoice_line il on i.invoice_id = il.invoice_id
    group by i.billing_country
)
select 
    cs.billing_country,
    cs.total_sales,
    cs.track_count,
    case 
        when cs.billing_country in ('China', 'India', 'Japan') then 'Asia'
        when cs.billing_country in ('Germany', 'France', 'UK') then 'Europe'
        when cs.billing_country in ('USA', 'Canada', 'Mexico') then 'North America'
        else 'Other'
    end as continent
from country_sales cs;
"""

In [62]:
df = pd.read_sql(query, conn)

In [63]:
conn.close()

In [64]:
continent_colors = {
    'Asia': 'yellow',
    'Europe': 'blue',
    'North America': 'red',
    'Other': 'gray'
}

In [65]:
fig = px.scatter(df, 
                 x='track_count', 
                 y='total_sales', 
                 color='continent', 
                 color_discrete_map=continent_colors,
                 title="Соотношение суммы продаж и количества треков по странам и континентам")


fig.update_layout(
    title_font_size=30, 
    title_font_color='gray',
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    xaxis_tickfont_size=14,
    yaxis_tickfont_size=14,
    xaxis_tickfont_color='gray',
    yaxis_tickfont_color='gray'
)

fig.show()

**Задача №2**

In [80]:
connection_url = URL.create(
    drivername="postgresql+psycopg2",
    username="postgres",
    password="123",  
    host="localhost",
    port=5432,
)

engine = create_engine(connection_url)
conn = engine.connect()

print("Успешное подключение к PostgreSQL!")

Успешное подключение к PostgreSQL!


In [81]:
query = """
select 
    genre_id, 
    count(track_id) as track_count
from track t
group by genre_id;
"""

In [82]:
df = pd.read_sql(query, conn)


In [78]:
conn.close()

In [84]:
df['color'] = 'gray' 
highlighted_genres = ['Jazz', 'Rock', 'Alternative & Punk']


df.loc[df['genre_id'].isin(highlighted_genres), 'color'] = df['genre_id'].map({
    'Jazz': 'blue', 
    'Rock': 'red', 
    'Alternative & Punk': 'green'
})

In [86]:
fig = px.bar(df, 
             x='genre_id', 
             y='track_count', 
             color='color', 
             color_discrete_map='identity',  
             title="Количество треков по жанрам")

fig.update_layout(
    title_font_size=30, 
    title_font_family="italic",
    title_font_color='gray', 
    plot_bgcolor='white', 
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    xaxis_tickfont_size=14,
    yaxis_tickfont_size=14,
    xaxis_tickfont_color='gray',
    yaxis_tickfont_color='gray',
    showlegend=False 
)

fig.show()